<a href="https://colab.research.google.com/github/SunStone-Secure-LLC/artemis-fedramp20x/blob/main/utils/Review_KSI_SAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from google.colab import files
import io

def generate_findings_summary(oscal_results_data):
    """
    Extracts target ID, title, and status from findings in OSCAL assessment results.

    Args:
        oscal_results_data (dict): Parsed OSCAL assessment results JSON data.

    Returns:
        list: A list of dictionaries, where each dictionary contains
              'target_id', 'title', and 'status' for a finding's target.
    """
    findings_summary = []

    # Navigate to the 'results' list
    results_list = oscal_results_data.get('assessment-results', {}).get('results', [])

    for result_item in results_list:
        # Navigate to the 'findings' list within each result item
        findings_list = result_item.get('findings', [])
        if not findings_list:
            print(f"No findings found in result item with UUID: {result_item.get('uuid')}")
            continue

        for finding in findings_list:
            target_info = finding.get('target')
            if not target_info:
                print(f"No target found for finding with UUID: {finding.get('uuid')}")
                continue

            target_id = target_info.get('target-id')
            # The title for the objective status is directly under the target object
            target_title = target_info.get('title')

            status_object = target_info.get('status', {})
            status_state = status_object.get('state')

            if target_id and target_title and status_state:
                findings_summary.append({
                    "target_id": target_id,
                    "title": target_title,
                    "status": status_state
                })
            else:
                missing_parts = []
                if not target_id: missing_parts.append("target-id")
                if not target_title: missing_parts.append("target title")
                if not status_state: missing_parts.append("status state")
                print(f"Incomplete target information for finding UUID {finding.get('uuid')}: Missing {', '.join(missing_parts)}")

    return findings_summary

# --- Main execution for Colab ---

# 1. Upload the OSCAL Assessment Results JSON file
print("Please upload your OSCAL Assessment Results JSON file.")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded. Exiting.")
else:
    # Assuming only one file is uploaded, get its name and content
    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]
    print(f"\nUploaded file: {file_name}")

    try:
        # Decode the bytes to a string and then parse JSON
        oscal_results_string = file_content.decode('utf-8')
        oscal_data = json.loads(oscal_results_string)

        # Generate the summary
        summary = generate_findings_summary(oscal_data)

        if summary:
            # Define the output file name
            output_file_name = 'findings_summary.json'

            # Save the summary to a JSON file
            with open(output_file_name, 'w') as f:
                json.dump(summary, f, indent=2)

            print(f"\nSuccessfully extracted {len(summary)} findings.")
            print(f"Summary saved to '{output_file_name}'. You can download it from the Colab file browser.")

            # Optional: Offer to download the file directly
            # files.download(output_file_name)

            # Print the summary for quick review
            print("\n--- Findings Summary ---")
            for item in summary:
                print(f"  Target ID: {item['target_id']}, Title: {item['title']}, Status: {item['status']}")
            print("------------------------")

        else:
            print("\nNo findings could be summarized. Please check the input file structure or content.")

    except json.JSONDecodeError:
        print("\nError: Invalid JSON file. Please ensure the uploaded file is a valid JSON document.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

Please upload your OSCAL Assessment Results JSON file.


Saving assessment-result-KSI-CNA.json to assessment-result-KSI-CNA.json

Uploaded file: assessment-result-KSI-CNA.json

Successfully extracted 8 findings.
Summary saved to 'findings_summary.json'. You can download it from the Colab file browser.

--- Findings Summary ---
  Target ID: CNA-DOS-M1, Title: CNA-DOS-M1 Status, Status: satisfied
  Target ID: CNA-BTC-M1, Title: CNA-BTC-M1 Status, Status: satisfied
  Target ID: CNA-ILP-M1, Title: CNA-ILP-M1 Status, Status: satisfied
  Target ID: CNA-MSE-M1, Title: CNA-MSE-M1 Status, Status: satisfied
  Target ID: CNA-NCE-M1, Title: CNA-NCE-M1 Status, Status: satisfied
  Target ID: CNA-CSC-M1, Title: CNA-CSC-M1 Status, Status: not-satisfied
  Target ID: CNA-HAR-M1, Title: CNA-HAR-M1 Status, Status: satisfied
  Target ID: KSI-CNA-AGGREGATE, Title: KSI-CNA-AGGREGATE Status, Status: satisfied
------------------------
